# Proof of concept search module

The idea with this module is to Google search 100+ results, then use an LLM to organize and re-rank the results.

## Ideas to consider

- Multiple searches: The company, company + product, search for product reviews, search all time vs just recent
- Provide a markdown template to fill out with an "other" category, and refine the "other" listings

In [1]:
from core import CompanyProduct, init_requests_cache, init_langchain_cache, make_experiment_dir

init_requests_cache()
init_langchain_cache()

'/home/keith/company-detective/.cache/langchain.sqlite'

In [2]:
target = CompanyProduct("Singularity 6", "Palia")
experiment_dir = make_experiment_dir(target)

In [3]:
from general_search import search_web, summarize, result_to_markdown

pages = search_web(target)
summary = summarize(target, pages)

print(summary)


with open(f"{experiment_dir}/search.md", "w") as f:
    f.write(summary.content)

    f.write("\n# Sources\n")
    for result in pages:
        f.write(result_to_markdown(result) + "\n\n")

2024-08-19 13:05:13.647 | DEBUG    | search:search:68 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - "Singularity 6"', 'totalResults': '55400', 'searchTerms': '"Singularity 6"', 'count': 10, 'startIndex': 1, 'language': 'la

content="# Official social media\n- [Singularity 6 (@Singularity6) / X](https://twitter.com/singularity6?lang=en)\n- [Singularity 6 | LinkedIn](https://www.linkedin.com/company/singularity6)\n\n# Job boards\n- [Singularity 6 Jobs - GameJobs.co](https://gamejobs.co/search?c=Singularity+6)\n- [Singularity 6 Los Angeles Office: Careers, Perks + Culture | Built In](https://www.builtinla.com/company/singularity-6)\n\n# App stores\n- [Palia on Steam](https://store.steampowered.com/app/2707930/Palia/)\n- [Palia for Nintendo Switch - Nintendo Official Site](https://www.nintendo.com/us/store/products/palia-switch/)\n- [Palia | Download and Play for Free - Epic Games Store](https://store.epicgames.com/en-US/p/palia-0d428e)\n\n# Product reviews\n- [Palia is a cozy, pleasant, and positive-natured MMO | Hands-on Review - God is a Geek](https://www.godisageek.com/2023/08/palia-is-a-cozy-pleasant-and-positive-natured-mmo-hands-on-preview/) (Aug 4, 2023)\n- [Early access life sim Palia has become my S

In [4]:
# # Test a few different URL compression schemes

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# for url in [
#     "https://www.apollo.io/companies/Pomelo-Care/6196af0887796a008c77f450",
#     "cache://www.apollo.io/1",
#     "cache://www.apollo.io/15",
#     "cache://apollo/15",
#     "http://apollo/15",
#     ]:
#     print(f"{llm.get_num_tokens(url):,} tokens: {url}")